##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Armazenamento de Blobs do Azure com o TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/azure"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/io/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Atenção: além dos pacotes Python, este notebook usa `npm install --user` para instalar pacotes. Tenha cuidado ao executar localmente.


## Visão geral

Este tutorial mostra como usar arquivos de leitura e gravação no [Armazenamento de Blobs do Azure](https://azure.microsoft.com/en-us/services/storage/blobs/) com o TensorFlow por meio da integração do sistema de arquivos do Azure com o TensorFlow IO.

É necessário ter uma conta de armazenamento do Azure para ler e gravar arquivos no Armazenamento de Blobs do Azure. A Chave de Armazenamento do Azure deve ser fornecida por meio da variável de ambiente:

```
os.environ['TF_AZURE_STORAGE_KEY'] = '&lt;key&gt;'
```

O nome da conta de armazenamento e o nome do container fazem parte do URI do nome do arquivo:

```
azfs://&lt;storage-account-name&gt;/&lt;container-name&gt;/&lt;path&gt;
```

Neste tutorial, para fins de demonstração, você pode configurar opcionalmente o [Azurite](https://github.com/Azure/Azurite), que é um emulador do Armazenamento do Azure. Com o emulador Azurite, é possível ler e gravar arquivos por meio da interface de  Armazenamento de Blobs do Azure pelo TensorFlow.

## Configuração e uso

### Instale os pacotes necessários e reinicie o runtime:

In [2]:
try:
  %tensorflow_version 2.x 
except Exception:
  pass

!pip install tensorflow-io

TensorFlow 2.x selected.
     |████████████████████████████████| 20.1MB 42.7MB/s 


### Instale e configure o Azurite (opcional)

Caso uma Conta de Armazenamento do Azure não esteja disponível, o código abaixo é necessário para instalar e executar o Azurite, que emula a interface do Armazenamento do Azure:

In [3]:
!npm install azurite@2.7.0

npm WARN deprecated request@2.87.0: request has been deprecated, see https://github.com/request/request/issues/3142
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ azurite@2.7.0
added 116 packages from 141 contributors in 6.591s


In [4]:
# The path for npm might not be exposed in PATH env,
# you can find it out through 'npm bin' command
npm_bin_path = get_ipython().getoutput('npm bin')[0]
print('npm bin path: ', npm_bin_path)

# Run `azurite-blob -s` as a background process. 
# IPython doesn't recognize `&` in inline bash cells.
get_ipython().system_raw(npm_bin_path + '/' + 'azurite-blob -s &')

npm bin path:  /content/node_modules/.bin


### Leitura e gravação de arquivos no Armazenamento do Azure com o TensorFlow

Veja abaixo um exemplo de leitura e gravação de arquivos no Armazenamento do Azure utilizando a API do TensorFlow.

Seu comportamento é igual ao de outros sistemas de arquivos (como POSIX ou GCS) no TensorFlow após a importação do pacote `tensorflow-io`, pois `tensorflow-io` vai registrar automaticamente o esquema `azfs` para uso.

A Chave de Armazenamento do Azure deve ser fornecida por meio da variável de ambiente `TF_AZURE_STORAGE_KEY`. Ou então `TF_AZURE_USE_DEV_STORAGE` pode ser definida como `True` para usar o emulador do Azurite:


In [ ]:
import os
import tensorflow as tf
import tensorflow_io as tfio

# Switch to False to use Azure Storage instead:
use_emulator = True

if use_emulator:
  os.environ['TF_AZURE_USE_DEV_STORAGE'] = '1'
  account_name = 'devstoreaccount1'
else:
  # Replace <key> with Azure Storage Key, and <account> with Azure Storage Account
  os.environ['TF_AZURE_STORAGE_KEY'] = '<key>'
  account_name = '<account>'

  # Alternatively, you can use a shared access signature (SAS) to authenticate with the Azure Storage Account
  os.environ['TF_AZURE_STORAGE_SAS'] = '<your sas>'
  account_name = '<account>'

In [6]:
pathname = 'az://{}/aztest'.format(account_name)
tf.io.gfile.mkdir(pathname)

filename = pathname + '/hello.txt'
with tf.io.gfile.GFile(filename, mode='w') as w:
  w.write("Hello, world!")

with tf.io.gfile.GFile(filename, mode='r') as r:
  print(r.read())

Hello, world!


## Configurações

As configurações do Armazenamento de Blobs do Azure no TensorFlow são sempre feitas por meio de variáveis de ambiente. Veja abaixo a lista completa de configurações disponíveis:

- `TF_AZURE_USE_DEV_STORAGE`: defina como 1 para usar o emulador de armazenamento de desenvolvimento local para conexões como 'az://devstoreaccount1/container/file.txt', que terá prioridade sobre todas as outras configurações. Portanto, utilize `unset` para usar qualquer outra conexão.
- `TF_AZURE_STORAGE_KEY`: chave da conta de armazenamento em uso.
- `TF_AZURE_STORAGE_USE_HTTP`: defina como qualquer valor se você não quiser usar transferência HTTPS. Utilize `unset` para usar o padrão HTTPS.
- `TF_AZURE_STORAGE_BLOB_ENDPOINT`: defina como o endpoint do armazenamento de blobs. O padrão é `.core.windows.net`.
